In [2]:
import pandas as pd
df_data = pd.read_csv("/kaggle/input/imdb-title-with-person-name/final_rating_table/final_rating_table.tsv",sep='\t')
df_data.head()

,tconst,averageRating,language,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres,director,editor,writer,actor_actress
0,tt0000005,6.2,en,short,Blacksmith Scene,0,1893,1,"Comedy,Short",william k.l. dickson,Unknown,Unknown,"charles kayser,john ott"
1,tt0000010,6.9,sv,short,Leaving the Factory,0,1895,1,"Documentary,Short",louis lumière,Unknown,Unknown,Unknown
2,tt0000012,7.4,tr,short,The Arrival of a Train,0,1896,1,"Documentary,Short","auguste lumière,louis lumière",Unknown,Unknown,Unknown
3,tt0000013,5.7,en,short,The Photographical Congress Arrives in Lyon,0,1895,1,"Documentary,Short",louis lumière,Unknown,Unknown,Unknown
4,tt0000014,7.1,sv,short,The Waterer Watered,0,1895,1,"Comedy,Short",louis lumière,Unknown,Unknown,"françois clerc,benoît duval"


In [3]:
print((df_data["startYear"]=="Other").sum())
print(df_data["startYear"])

0
0         1893
1         1895
2         1896
3         1895
4         1895
          ... 
258022    2020
258023    2019
258024    2019
258025    2019
258026    2019
Name: startYear, Length: 258027, dtype: object


In [4]:
df_data.replace('Unknown', 'Other', inplace=True)
print((df_data["writer"]=="Other").sum())

103516


In [5]:
df_data['runtimeMinutes'] = pd.to_numeric(df_data['runtimeMinutes'], errors='coerce')
df_data['startYear'] = pd.to_numeric(df_data['startYear'], errors='coerce')

In [6]:
df_data.head()

,tconst,averageRating,language,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres,director,editor,writer,actor_actress
0,tt0000005,6.2,en,short,Blacksmith Scene,0,1893.0,1.0,"Comedy,Short",william k.l. dickson,Other,Other,"charles kayser,john ott"
1,tt0000010,6.9,sv,short,Leaving the Factory,0,1895.0,1.0,"Documentary,Short",louis lumière,Other,Other,Other
2,tt0000012,7.4,tr,short,The Arrival of a Train,0,1896.0,1.0,"Documentary,Short","auguste lumière,louis lumière",Other,Other,Other
3,tt0000013,5.7,en,short,The Photographical Congress Arrives in Lyon,0,1895.0,1.0,"Documentary,Short",louis lumière,Other,Other,Other
4,tt0000014,7.1,sv,short,The Waterer Watered,0,1895.0,1.0,"Comedy,Short",louis lumière,Other,Other,"françois clerc,benoît duval"


In [7]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258027 entries, 0 to 258026
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          258027 non-null  object 
 1   averageRating   258027 non-null  float64
 2   language        258027 non-null  object 
 3   titleType       258027 non-null  object 
 4   primaryTitle    258027 non-null  object 
 5   isAdult         258027 non-null  int64  
 6   startYear       258003 non-null  float64
 7   runtimeMinutes  195847 non-null  float64
 8   genres          258027 non-null  object 
 9   director        258027 non-null  object 
 10  editor          258027 non-null  object 
 11  writer          258027 non-null  object 
 12  actor_actress   258027 non-null  object 
dtypes: float64(3), int64(1), object(9)
memory usage: 25.6+ MB


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [9]:
fearures = ['language', 'titleType', 'isAdult', 'startYear', 'runtimeMinutes', 'genres', 'director', 'editor', 'writer', 'actor_actress']
targwt = 'averageRating'

train_df, test_df = train_test_split(df_data, test_size=0.2, random_state=42)
#after try with shuffle=True

In [10]:
# Preprocessing pipeline
numeric_features = ['isAdult', 'runtimeMinutes', 'startYear']
categorical_features = ['language', 'titleType', 'genres', 'director', 'editor', 'writer', 'actor_actress']
# datetime_features = ['startYear']

In [11]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# datetime_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),  # Adjust imputation strategy if needed
# ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
#         ('datetime', datetime_transformer, datetime_features)
    ])

In [12]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, verbose=1))
])

In [ ]:
model.fit(train_df[fearures], train_df[targwt])

In [ ]:
import joblib
joblib.dump(model, 'random_forest_model.joblib')

In [ ]:
# Make predictions
predictions = model.predict(test_df[fearures])

# Evaluate the model
mse = mean_squared_error(test_df[targwt], predictions)
print(f'Mean Squared Error: {mse}')

In [ ]:

# To load the model back
# loaded_model = joblib.load('random_forest_model.joblib')